# Mushroom Toxicity Classifier (Keras + EfficientNetB0)

This notebook trains a binary image classifier to predict whether a mushroom is toxic (1) or edible (0) using transfer learning with EfficientNetB0.

It includes:

- Automatic Kaggle download of a mushroom images dataset (MO-106, 94 species, ~27k images).
- A species->toxicity mapping step to build a binary folder-of-folders dataset (edible/, toxic/).
- A tf.data pipeline with AUTOTUNE for performance.
- EfficientNetB0 feature extraction + fine-tuning.
- Class weights and threshold tuning to reduce false edible on toxic images.
- (Optional) a tiny scratch CNN baseline for comparison.

> References

> - EfficientNet transfer learning (Keras official example): https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

> - EfficientNetB0 API docs: https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB0

> - Kaggle MO-106 (94 species): https://www.kaggle.com/datasets/iftekhar08/mo-106


In [ ]:
# Optional: install/upgrade packages if needed
# !pip install -U tensorflow scikit-learn kaggle

import os, glob, shutil, zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
print('TensorFlow:', tf.__version__)


## 1) Kaggle setup (one-time)

This notebook uses Kaggle CLI to download the dataset. You need an API token at `~/.kaggle/kaggle.json`.

How to obtain and place it:

1. Go to https://www.kaggle.com/ -> Profile -> Account -> Create New API Token (downloads kaggle.json).

2. Upload it to this runtime at `~/.kaggle/kaggle.json` and set permissions `0o600`.

Uncomment and run the cell below if you need to place the token programmatically.


In [ ]:
# # PLACE YOUR KAGGLE TOKEN (paste the JSON content between triple quotes)
# kaggle_token = '''
# {"username":"YOUR_NAME","key":"YOUR_KEY"}
# '''
# import os
# os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
# with open(os.path.expanduser('~/.kaggle/kaggle.json'), 'w') as f:
#     f.write(kaggle_token.strip())
# os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)


## 2) Download & unpack MO-106 (images) from Kaggle

We will download MO-106 (94 species, ~27k images) and unpack under `data/mo106_raw/`.


In [ ]:
DATA_DIR = 'data'
RAW_DIR  = os.path.join(DATA_DIR, 'mo106_raw')
os.makedirs(DATA_DIR, exist_ok=True)
print('DATA_DIR:', DATA_DIR)

# Download (uncomment in a real runtime with Kaggle CLI available)
# !pip -q install kaggle
# !kaggle datasets download -d iftekhar08/mo-106 -p {DATA_DIR} -o

# Unzip (if needed)
# for z in glob.glob(os.path.join(DATA_DIR, '*.zip')):
#     print('Unzipping:', z)
#     with zipfile.ZipFile(z, 'r') as f:
#         f.extractall(RAW_DIR)

print('Expect raw dataset under:', RAW_DIR)


## 3) Build a binary dataset (edible vs toxic)

MO-106 is organized by species. We need to map each species to edible or toxic and then copy images into two folders. Expected structure after this step:

- data/mushrooms_binary/
  - edible/
  - toxic/

Important: MO-106 variants may include a CSV/metadata file with edibility. Prefer using the official metadata when present. The code below shows a fallback heuristic (looks for tokens like 'edible', 'toxic', 'poison' in folder names). If your copy includes a reliable CSV, load it and replace the heuristic mapping.


In [ ]:
BIN_DIR = os.path.join(DATA_DIR, 'mushrooms_binary')
if os.path.exists(BIN_DIR):
    shutil.rmtree(BIN_DIR)
os.makedirs(os.path.join(BIN_DIR, 'edible'), exist_ok=True)
os.makedirs(os.path.join(BIN_DIR, 'toxic'),  exist_ok=True)

species_to_label = {}
if os.path.isdir(RAW_DIR):
    for species_dir in glob.glob(os.path.join(RAW_DIR, '*')):
        if not os.path.isdir(species_dir):
            continue
        folder = os.path.basename(species_dir)
        name   = folder.lower()
        if ('toxic' in name) or ('poison' in name):
            species_to_label[folder] = 'toxic'
        elif 'edible' in name:
            species_to_label[folder] = 'edible'
        else:
            species_to_label[folder] = 'unknown'

print('Species mapped:', len(species_to_label))
stats = {'edible':0, 'toxic':0, 'skipped':0}
for species_dir in glob.glob(os.path.join(RAW_DIR, '*')):
    if not os.path.isdir(species_dir):
        continue
    tag = species_to_label.get(os.path.basename(species_dir), 'unknown')
    if tag not in ('edible','toxic'):
        stats['skipped'] += 1
        continue
    target = os.path.join(BIN_DIR, tag)
    for img in glob.glob(os.path.join(species_dir, '*')):
        if img.lower().endswith(('.jpg','.jpeg','.png')):
            shutil.copy(img, target)
            stats[tag] += 1

print('Binary copy stats:', stats)
print('Binary root:', BIN_DIR)


## 4) tf.data pipeline (with AUTOTUNE)

We build a train/validation split directly from the `edible/` and `toxic/` folders.


In [ ]:
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32
AUTOTUNE   = tf.data.AUTOTUNE

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    BIN_DIR, validation_split=0.2, subset='training', seed=42,
    label_mode='binary', image_size=IMG_SIZE, batch_size=BATCH_SIZE)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    BIN_DIR, validation_split=0.2, subset='validation', seed=42,
    label_mode='binary', image_size=IMG_SIZE, batch_size=BATCH_SIZE)

normalize = layers.Rescaling(1./255)
augment   = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
])

def prep(x, y, training=False):
    x = normalize(x)
    if training:
        x = augment(x)
    return x, y

train_ds = train_ds.map(lambda x,y: prep(x,y,True), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds   = val_ds.map(prep, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

# Compute simple class weights
counts = {'edible':0, 'toxic':0}
for _, y in train_ds.unbatch().take(20000):
    counts['toxic' if int(y.numpy())==1 else 'edible'] += 1

total = max(1, counts['edible'] + counts['toxic'])
w_edible = total / (2.0 * max(1, counts['edible']))
w_toxic  = total / (2.0 * max(1, counts['toxic']))
class_weights = {0: w_edible, 1: w_toxic}
print('Class counts:', counts)
print('Class weights:', class_weights)


## 5) EfficientNetB0 (feature extraction -> fine-tuning)

We follow Keras' transfer learning pattern: freeze the base (ImageNet weights), train the head, then unfreeze top blocks and fine-tune with a smaller learning rate.


In [ ]:
from tensorflow.keras.applications import EfficientNetB0

base = EfficientNetB0(include_top=False, weights='imagenet', input_shape=IMG_SIZE+(3,))
base.trainable = False

inputs = layers.Input(shape=IMG_SIZE+(3,))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs, name='mushroom_effnet_b0')

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy', metrics=['accuracy'])

history_fe = model.fit(train_ds, validation_data=val_ds, epochs=5, class_weight=class_weights)

base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

history_ft = model.fit(train_ds, validation_data=val_ds, epochs=5, class_weight=class_weights)


## 6) Threshold tuning (prioritize toxic recall)

We sweep decision thresholds and pick one with high recall on the toxic class.


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

y_true, y_prob = [], []
for xb, yb in val_ds:
    y_true.append(yb.numpy().ravel())
    y_prob.append(model.predict(xb, verbose=0).ravel())
y_true = np.concatenate(y_true)
y_prob = np.concatenate(y_prob)

cands = np.linspace(0.30, 0.80, 21)
records = []
for th in cands:
    y_pred = (y_prob >= th).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    records.append((th, p, r, f1, cm))

best = sorted(records, key=lambda x: (x[2], x[1], x[3]))[-1]
print('Best threshold:', round(best[0],2))
print('Precision:', round(best[1],3), 'Recall:', round(best[2],3), 'F1:', round(best[3],3))
print('Confusion matrix:
', best[4])


## 7) (Optional) Tiny CNN baseline (from scratch)

A compact CNN to visualize the lift you get from transfer learning.


In [ ]:
def tiny_cnn(input_shape=IMG_SIZE+(3,)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.25)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs, name='tiny_cnn')

baseline = tiny_cnn()
baseline.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# baseline.fit(train_ds, validation_data=val_ds, epochs=6, class_weight=class_weights)


## Appendix — What is EfficientNetB0 (in brief)?

EfficientNetB0 is the baseline model in the EfficientNet family. It achieves a strong accuracy-efficiency trade-off by compound scaling of depth, width, and resolution. It uses MBConv blocks, depthwise separable convolutions, and squeeze-and-excitation modules.

- Great for transfer learning on small/medium datasets.

- Input size: 224x224x3.

- Docs: https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB0

- Keras example: https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
